In [722]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *
from _rulesbuilding import *

In [723]:
rules = pd.read_csv(config.inputs['rules']['fullpath'])
rules = rules[(rules['text_match'].notna()) & (rules['text_exclude'].notna() == False)]

regexer = GroupedDataRegex('|')

builder = RulesRegexBuilder(GroupedDataRegex('|'))

builder.appendDecorator(RegexDecorator('M', '(', ')'))
builder.appendDecorator(RegexDecorator('S', '^(', ').*$'))
builder.appendDecorator(RegexDecorator('A', '^.*(', ').*$'))

regexes_dict = builder.build(rules) 

{'^(Lim Commercials 3mo|CBS - Fake Cancel|CBS All Access|Commercial Free 1 Week|CBS - Fake New|CBS - Fake Cancel|Cancellation Confirmation CBS|CBS - Fake Cancel Passive Churn).*$': 39,
 '^(Netflix|Basic (Netflix)|Premium (Netflix)|Standard (Netflix)|Netflix - Fake Cancel Passive Churn|Netflix).*$': 3,
 '^(SHOWTIME|SHOWTIME - Fake New|SHOWTIME|Cancellation Confirmation SHOWTIME|SHOWTIME - Fake Cancel|SHOWTIME - Fake Cancel|CBS All Access and Showtime|Showtime 1 month free trial|Showtime|SHOWTIME - Fake New|SHOWTIME - Fake Cancel).*$': 12,
 '^(STARZ|com.starz.starzplay.subscriptions|STARZ - Fake Cancel|STARZ|STARZ - Fake New|STARZ - Fake Cancel Passive Churn|STARZ - Fake New|STARZ - Fake Cancel).*$': 8,
 '^(hulu|Hulu|Renewable Subscription iOS|Disney Bundle: Disney+, Hulu, and ESPN+|Renewable Subscription Apple TV|Hulu - Fake New current subscriber).*$': 1,
 '^.*(All Access|CBS).*$': 39,
 '^.*(Disney bundle subscription has been canceled).*$': 1,
 '^.*(Netflix).*$': 3,
 '^.*(STARZ|Starz|

In [724]:
signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.inputs['signals'].items()}

In [725]:
data = pd.read_csv(config.inputs['data']['fullpath'])

In [726]:
data = data[data['status'] != 'D']

In [727]:
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [728]:
data['service_id'] = data[['description']]
data['service_id'] = data[['service_id']].replace({'service_id':regexes_dict}, regex=True)

In [729]:
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

In [730]:
save_file(config.outputs['local']['fullpath'], data.to_csv())

In [731]:
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket(config.outputs['cloud']['bucket_name'])
blob = bucket.blob(config.outputs['local']['filename'])
blob.upload_from_filename(config.outputs['local']['fullpath'])

output(blob.public_url)

'https://storage.googleapis.com/antenna-task/processed_data.csv'


In [732]:
# print(data.to_string())